In [331]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2
import scipy
import math
import pickle


#Scaling
normilize = True
scale = False

#Spliting features and classifiers
SUBSET_FEATURES = False
SUBMODELS = False

#Showing Each classifier
SEPERATE_CLASSIFIERS = True

#Combining output
NN_COMBINE = True
VOTING_CLASSIFIERS = True


pd.set_option('display.max_columns', None)

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()



sql = "select   fights.f1result, \
                f1.height_inches AS f1_height,\
                f1.reach_inches AS f1_reach, \
                f1.weight_lbs AS f1_weight, \
                f1.strike_offense_per_min AS f1_strike_offense_per_min, \
                f1.strike_offense_accuracy AS f1_strike_offense_accuracy, \
                f1.strike_defense_per_min AS f1_strike_defense_per_min, \
                f1.strike_defense_accuracy AS f1_strike_defense_accuracy, \
                f1.takedowns_per_fight AS f1_takedowns_per_fight, \
                f1.takedowns_accuracy AS f1_takedowns_accuracy, \
                f1.takedowns_defense AS f1_takedowns_defense, \
                f1.submissions_per_fight AS f1_submissions_per_fight, \
                f1.total_fights AS f1_total_fights, \
                f1.association AS f1_association, \
                f1.country AS f1_country, \
                f1.wins AS f1_wins, \
                f1.losses AS f1_losses, \
                f1.total_fights AS f1_total_fights, \
                                                \
                f2.height_inches AS f2_height, \
                f2.reach_inches AS f2_reach, \
                f2.weight_lbs AS f2_weight, \
                f2.strike_offense_per_min AS f2_strike_offense_per_min, \
                f2.strike_offense_accuracy AS f2_strike_offense_accuracy, \
                f2.strike_defense_per_min AS f2_strike_defense_per_min, \
                f2.strike_defense_accuracy AS f2_strike_defense_accuracy, \
                f2.takedowns_per_fight AS f2_takedowns_per_fight, \
                f2.takedowns_accuracy AS f2_takedowns_accuracy, \
                f2.takedowns_defense AS f2_takedowns_defense, \
                f2.submissions_per_fight AS f2_submissions_per_fight, \
                f2.total_fights AS f2_total_fights, \
                f2.association AS f2_association, \
                f2.country AS f2_country, \
                f2.wins AS f2_wins, \
                f2.losses AS f2_losses, \
                f2.total_fights AS f2_total_fights \
                from octagon.fights INNER JOIN octagon.fighters AS f1 ON octagon.fights.f1name = f1.name INNER JOIN octagon.fighters AS f2 ON octagon.fights.f2name = f2.name;"

df = pd.read_sql_query(sql, conn)

test = pd.read_sql_query("select * from octagon.fighters",conn)



# df = pd.read_sql_query("select * from octagon.fights",conn)
df2 = pd.read_sql_query("select * from octagon.fighters",conn)
conn.close()


In [332]:
len(test['country'].unique())
# print(test['reach_inches'].isnull().sum())
# test = test.interpolate()
# test = test.dropna()
# test.head()
# print(test['reach_inches'].isnull().sum())

63

### Convert text ones into numeric with normilization

In [333]:


country_names = list(test['country'].unique())
df['f1_country'] = df['f1_country'].apply(lambda x: country_names.index(x) / len(country_names))
df['f2_country'] = df['f2_country'].apply(lambda x: country_names.index(x) / len(country_names))


asso_names = list(test["association"].unique())
df['f1_association'] = df['f1_association'].apply(lambda x: asso_names.index(x) / len(asso_names))
df['f2_association'] = df['f2_association'].apply(lambda x: asso_names.index(x) / len(asso_names))


with open("text_based_names.pickle","wb") as f:
    pickle.dump([country_names,asso_names],f)

# df['f1_association'] = df['f1_association'].apply(lambda x: names.index(x))
# df['f2_association'] = df['f2_association'].apply(lambda x: names.index(x))

df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,win,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.264984,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3
1,win,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.694006,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25
2,win,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.001577,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3
3,win,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004732,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2
4,win,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.728707,0.079365,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1


In [334]:
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])
df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,1,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.264984,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3
1,1,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.694006,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25
2,1,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.001577,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3
3,1,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004732,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2
4,1,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.728707,0.079365,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1


### Subsetting the features to improve performance

In [335]:
if SUBSET_FEATURES:
    subset = ['f1result','f1_strike_offense_per_min','f1_strike_defense_per_min','f1_association','f1_wins','f1_losses', 
        'f2_strike_offense_per_min','f2_strike_defense_per_min','f2_association','f2_wins','f2_losses']
    df = df[subset]

### Interpolate the data to fill in the NAs to get a better result of the data
It was found that dropna works the best for the highest correclation results, We will revisit this later

In [336]:
df0 = df.interpolate()
df0 = df.interpolate(method='spline', order=2)
df = df.interpolate(method='pchip')
df = df.dropna()

df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,1,78,81.000000,260,2.820000,34.00,2.460000,46.000000,3.260000,66.000000,50.000000,2.200000,4,0.264984,0.000000,1,3,4,79,81.000000,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3
1,1,70,77.995370,151,2.474875,35.75,3.065417,47.984496,1.316667,32.845745,52.948718,1.964773,7,0.694006,0.000000,4,3,7,70,73.000000,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25
2,1,72,75.657407,170,1.890000,36.00,3.230000,52.000000,0.620000,16.000000,58.000000,1.500000,3,0.001577,0.000000,1,2,3,68,66.000000,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3
3,1,69,74.000000,247,0.970000,30.00,2.870000,59.000000,1.060000,12.000000,66.000000,0.700000,7,0.004732,0.000000,4,3,7,74,68.851852,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2
4,1,72,73.000000,170,1.610000,41.00,2.000000,46.000000,4.170000,70.000000,45.000000,0.600000,9,0.728707,0.079365,4,5,9,74,74.148148,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1


## Move the lossers to the end to have data for losing people

In [337]:
df.corr()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
f1result,1.000000,-0.022554,-0.000957,-0.002347,-0.035534,-0.006344,-0.026795,-0.053046,0.005005,-0.021626,-0.019413,-0.034050,0.000873,-0.017101,0.004611,-0.001856,-0.025664,0.000873,-0.012382,0.006645,0.008947,-0.016441,-0.007903,-0.025129,-0.028475,0.019169,-0.006148,-0.005802,-0.019118,0.005481,0.001472,0.003959,0.000294,-0.020774,0.005481
f1_height,-0.022554,1.000000,0.757962,0.778365,-0.008917,0.164315,-0.054794,-0.156158,-0.088966,0.029064,-0.000416,0.031791,0.146511,0.009553,0.055783,0.141388,0.118318,0.146511,0.677611,0.513141,0.740942,-0.031835,0.139530,-0.007823,-0.111389,0.020558,0.014562,0.010102,0.013379,0.087581,-0.024090,0.055782,0.083720,0.080082,0.087581
f1_reach,-0.000957,0.757962,1.000000,0.641861,0.001195,0.226069,-0.097921,-0.184099,-0.023103,0.075851,0.030010,-0.001357,0.159192,-0.032140,0.022467,0.176343,0.081492,0.159192,0.548583,0.453748,0.618874,-0.041059,0.118584,-0.042073,-0.089497,0.021671,0.018111,0.019049,0.006247,0.110412,-0.026475,0.024447,0.111557,0.085968,0.110412
f1_weight,-0.002347,0.778365,0.641861,1.000000,-0.045359,0.199356,-0.057115,-0.210220,-0.046043,0.009644,0.032418,-0.068527,0.157982,-0.040654,0.049523,0.141487,0.139906,0.157982,0.730157,0.550808,0.927305,-0.050892,0.148832,-0.042841,-0.178493,0.000247,-0.003874,0.013058,0.008449,0.102430,-0.032116,0.061056,0.100185,0.083987,0.102430
f1_strike_offense_per_min,-0.035534,-0.008917,0.001195,-0.045359,1.000000,0.228125,0.358220,0.261276,-0.170289,0.049914,0.314302,-0.211637,0.013513,-0.019591,0.029296,0.093729,-0.119533,0.013513,-0.049717,-0.049463,-0.065854,0.212354,0.055984,0.175921,0.103140,0.016736,-0.020435,0.096701,-0.066141,0.043508,0.046782,0.018369,0.050778,0.024030,0.043508
f1_strike_offense_accuracy,-0.006344,0.164315,0.226069,0.199356,0.228125,1.000000,-0.157093,-0.105825,0.196134,0.175230,0.034745,0.034966,0.023539,0.025456,0.047781,0.076653,-0.078430,0.023539,0.143508,0.120408,0.188251,0.015281,0.128559,0.012686,-0.101413,0.007766,-0.021154,0.035829,-0.018076,0.037825,0.008492,0.016450,0.049478,0.010544,0.037825
f1_strike_defense_per_min,-0.026795,-0.054794,-0.097921,-0.057115,0.358220,-0.157093,1.000000,-0.248800,-0.345519,-0.213175,0.073308,0.002907,-0.098178,0.012723,0.015320,-0.143962,0.019325,-0.098178,-0.049391,-0.051932,-0.081860,0.167807,0.058232,0.115042,0.055735,-0.008619,-0.000062,0.009235,-0.040937,-0.062935,0.005121,0.019176,-0.072902,-0.029453,-0.062935
f1_strike_defense_accuracy,-0.053046,-0.156158,-0.184099,-0.210220,0.261276,-0.105825,-0.248800,1.000000,0.037574,0.144154,0.296166,-0.209177,0.133556,-0.079330,-0.013122,0.183170,0.009861,0.133556,-0.112594,-0.079186,-0.177138,0.048932,-0.109213,-0.020795,0.158696,0.020218,0.018541,0.066520,-0.000098,0.117334,0.004676,-0.018655,0.114645,0.102263,0.117334
f1_takedowns_per_fight,0.005005,-0.088966,-0.023103,-0.046043,-0.170289,0.196134,-0.345519,0.037574,1.000000,0.253898,-0.016473,0.108390,0.065976,-0.044934,-0.074423,0.102241,-0.027436,0.065976,-0.020565,-0.002002,-0.033796,-0.049118,0.048332,-0.081521,-0.004072,-0.014978,-0.021482,-0.015716,0.041034,0.032379,-0.005028,-0.025670,0.043154,0.005696,0.032379
f1_takedowns_accuracy,-0.021626,0.029064,0.075851,0.009644,0.049914,0.175230,-0.213175,0.144154,0.253898,1.000000,0.155473,-0.010655,0.120383,-0.018555,0.001808,0.153551,0.027319,0.120383,0.003516,0.030273,0.015166,-0.021101,-0.000110,-0.033984,0.020067,

In [338]:
f1_end = math.ceil(len(df.columns) / 2)
df2 = df.copy()
tmp = df2.ix[:,1:f1_end]


df2.ix[:,1:f1_end] = df2.ix[:,f1_end:].values
df2.ix[:,f1_end:] = tmp.values
df2['f1result'] = 0


df2.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,0,79,81.000000,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3,78,81.000000,260,2.820000,34.00,2.460000,46.000000,3.260000,66.000000,50.000000,2.200000,4,0.264984,0.000000,1,3,4
1,0,70,73.000000,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25,70,77.995370,151,2.474875,35.75,3.065417,47.984496,1.316667,32.845745,52.948718,1.964773,7,0.694006,0.000000,4,3,7
2,0,68,66.000000,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3,72,75.657407,170,1.890000,36.00,3.230000,52.000000,0.620000,16.000000,58.000000,1.500000,3,0.001577,0.000000,1,2,3
3,0,74,68.851852,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2,69,74.000000,247,0.970000,30.00,2.870000,59.000000,1.060000,12.000000,66.000000,0.700000,7,0.004732,0.000000,4,3,7
4,0,74,74.148148,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1,72,73.000000,170,1.610000,41.00,2.000000,46.000000,4.170000,70.000000,45.000000,0.600000,9,0.728707,0.079365,4,5,9


### Combine the two dataframes and randomize

In [339]:
df = df.append(df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,0,75,76.000000,185,3.780000,44.000000,2.990,57,0.29,19.0,59.000000,0.4,15,0.823344,0.000000,8,7,15,72,76.0,185,3.08,47,2.40,58,2.59,34,78,0.5,14,0.093060,0.015873,9,4,14
1,0,71,73.000000,170,2.584193,46.235294,3.465,54,1.03,36.5,69.554348,0.4,8,0.807571,0.000000,4,4,8,71,76.0,170,3.29,47,0.82,78,5.84,56,100,0.0,2,0.178233,0.158730,2,0,2
2,0,68,70.000000,155,3.850000,41.000000,3.470,60,1.25,65.0,63.000000,0.8,10,0.689274,0.047619,4,6,10,72,76.0,170,4.64,44,3.69,54,1.17,30,45,1.4,21,0.422713,0.000000,13,8,21
3,0,76,69.814815,265,1.850000,51.000000,1.150,61,2.30,39.0,40.000000,1.0,7,0.662461,0.000000,5,2,7,73,73.5,241,1.93,34,3.02,50,0.26,28,78,0.0,10,0.088328,0.015873,5,5,10
4,0,68,69.000000,155,3.540000,41.000000,3.570,64,0.86,38.0,75.000000,0.0,17,0.108833,0.222222,10,7,17,73,74.0,145,3.00,33,3.88,59,0.60,36,38,1.8,17,0.004732,0.000000,10,7,17


In [340]:
#Pull the labels out of the dataset
y = df['f1result']
del df['f1result']

X = df

In [341]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split
#MLP Classifer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.feature_selection import RFE

if normilize:
    X = preprocessing.normalize(X, norm='l2')
    
if scale:
    X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state = 1)

names = [
         "MLP",
         "Naive Bayes",
         "KNN",
         "SVM Linear",
         "SVM gamma",
         "Decsion Tree", 
         "Random Forest",
         "adaBoost"
        ]

clfs = [
        MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=400),
        GaussianNB(),
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025,probability=True),
        SVC(gamma=2, C=1,probability=True),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        AdaBoostClassifier()
       ]

if SUBMODELS and SUBSET_FEATURES:
    #feats = [0,2,3]
    feats = [0,2,3,4,5,6,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]
elif SUBMODELS:
    feat = [3,5,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]

#predict_proba to get the probabilities 
    
    #All Features
# SVC- Linear, DTree and AdaBoost are the best especially with Voter
 
#Subset
# MLP, KNN, SVM Linear 
# Normilization Good 
    
    
#13, 15, 16, 30, 
# print("NBC Accuracy of the model is: %.2f%%" % (accuracy_score(nbc_preds,y_test) * 100))
# print("MLP Accuracy of the model is: %.2f%%" % (accuracy_score(mlp_preds,y_test) * 100))


## Finding the importance of each feature using Recursion

In [342]:
if False:
    f1_end = int(len(df.columns) / 2)
    indexs = np.array([i for i in range(f1_end)])
    selector = RFE(clfs[3], 14, step=1)
    selector = selector.fit(X_train, y_train)
    print(selector.support_)
    print(indexs[selector.support_[:f1_end]])
    print(indexs[selector.support_[f1_end:]])
#

## Looking at each Classifier separatly 

In [343]:
if SEPERATE_CLASSIFIERS:
    
    pred_list_train = []
    pred_list_test = []
    
    for index, clf in enumerate(clfs):
        clf.fit(X_train,y_train)

        preds = clf.predict(X_test)
        
        preds_train = np.amax(clf.predict_proba(X_train),axis=1)
        preds_test = np.amax(clf.predict_proba(X_test),axis=1)
        
        pred_list_train.append(preds_train)
        pred_list_test.append(preds_test)
        #print(names[index] + " Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))
        print("%.2f" % (accuracy_score(preds,y_test) * 100))

    with open("clfs-2.pickle","wb") as f:
        pickle.dump(clfs,f)

50.42
68.25
63.50
60.44
72.38
74.90
69.55
75.36


## Using Voting for Combination

In [344]:
if VOTING_CLASSIFIERS:
    clf_vote = VotingClassifier(estimators=list(zip(names,clfs)),voting='soft')
    clf_vote.fit(X_train,y_train)
    preds = clf_vote.predict(X_test)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))


74.22


### Testing Combination of Classifiers using NN - DOES NOT WORK GOOD

In [345]:
if NN_COMBINE:
    combine_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    combine_clf.fit(np.array(pred_list_train).T,y_train)
    preds = combine_clf.predict(np.array(pred_list_test).T)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))



51.49


In [346]:

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()

sql = "select fid, \
            height_inches, \
            reach_inches, \
            weight_lbs, \
            strike_offense_per_min, \
            strike_offense_accuracy, \
            strike_defense_per_min, \
            strike_defense_accuracy, \
            takedowns_per_fight, \
            takedowns_accuracy, \
            takedowns_defense, \
            submissions_per_fight, \
            total_fights, \
            association, \
            country, \
            wins, \
            losses, \
            total_fights FROM octagon.fighters"

#16635
#230

#df1 = pd.read_sql_query(sql + str(16635), conn)
df = pd.read_sql_query("select * from octagon.fighters", conn)

conn.close()


In [347]:
#Figure out good interpolator without having to load the entire database each time
df = df.interpolate()

f1 = df[df['fid'] == 230].drop('fid',axis=1)
f2 = df[df['fid'] == 230].drop('fid',axis=1)

if not f1.empty and not f2.empty:
    print("hello")
#np.append(f1,f2)



hello
